
<h1 align="center">Cell Tracking according to Biological Needs</h1>
<h2 align="center">Strong Mitosis-aware Random-finite Sets Tracker with Aleatoric Uncertainty</h2>

<p align="center">
  <img alt="Teaser Image" src="./assets/framework.png">
  <br>
    <em>The output distribution of EmbedTrack using different input transformations on BF-C2DL-HSC.</em>
</p>

# Select the Dataset you want to process

In [2]:
# Select the dataset you want to process
DATASET = 'DIC-C2DH-HeLa'
SUBSET = 'train'
SEQUENCE = '01'

# Select some Inference Parameter
SHIFTS = 50

# Check if the selection is valid
AVAILABLE_DATASETS = [
    "Fluo-N2DH-SIM+",
    "Fluo-C2DL-MSC",
    "Fluo-N2DH-GOWT1",
    "PhC-C2DL-PSC",
    "BF-C2DL-HSC",
    "Fluo-N2DL-HeLa",
    "BF-C2DL-MuSC",
    "DIC-C2DH-HeLa",
    "PhC-C2DH-U373",
]
AVAILABLE_SUBSETS = ['train', 'challenge']
AVAILABLE_SEQUENCES = ['01', '02']
assert DATASET in AVAILABLE_DATASETS, f"Invalid dataset: {DATASET}"
assert SUBSET in AVAILABLE_SUBSETS, f"Invalid subset: {SUBSET}"
assert SEQUENCE in AVAILABLE_SEQUENCES, f"Invalid sequence: {SEQUENCE}"

# Setting up the environment

Creating the environment
  - downloading the CTC data
  - downloading the pretrained models. 

In [3]:
# Download the data
if SUBSET == 'train':
    !python utils/prepare_data.py --datasets $DATASET --train
else:
    !python utils/prepare_data.py --datasets $DATASET --challenge

In [4]:
# Download the pretrained models
import zipfile

!curl -O https://www.tnt.uni-hannover.de/de/project/MPT/data/BiologicalNeeds/models.zip
with zipfile.ZipFile("models.zip") as z:
    z.extractall("")

--2024-09-24 10:02:49--  https://www.tnt.uni-hannover.de/de/project/MPT/data/BiologicalNeeds/models.zip
Resolving www.tnt.uni-hannover.de (www.tnt.uni-hannover.de)... 130.75.31.71
Connecting to www.tnt.uni-hannover.de (www.tnt.uni-hannover.de)|130.75.31.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248308697 (237M) [application/zip]
Saving to: ‘models.zip’

models.zip          100%[===================>] 236.80M  61.9MB/s    in 3.8s    

2024-09-24 10:02:54 (61.9 MB/s) - ‘models.zip’ saved [248308697/248308697]



# Inference

## 1) EmbedTrack with Aleatoric Uncertainty by Advanced Test-Time Augmentation

In [8]:
import os
from embedtrack.scripts.inference import process

res_path = os.path.join(os.getcwd(), 'results/embedtrack')

process(
    model_dir=os.path.join(os.getcwd(), 'models'),
    res_dir=res_path,
    shifts=[0, SHIFTS],
    multiscale=False,
    multi_segmentation=False,
    batch_size=32,
    refine_segmentation=False,
    raw_data_paths=[os.path.join(os.getcwd(), "Data", "train")],
    datasets=[DATASET],
    sequences=[SEQUENCE],
)

## 2) Multi Hypothesis Tracking

In [ ]:
from mht.scripts.inference import infer_all

in_path = res_path
res_path = os.path.join(os.getcwd(), 'results/mht')

infer_all(
    data_root=in_path,
    dest_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
    tracker_args={},
)

## 3) Interpolation to fill temporal gaps in trajectories

In [ ]:
from utils.interpolate import postprocess_all

in_path = res_path
res_path = os.path.join(os.getcwd(), 'results/interpolated')

postprocess_all(
    data_root=in_path,
    dest_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
)

## 4) Postprocessing to satisfy the CTC requirements

In [ ]:
from utils.postprocess import postprocess_all

postprocess_all(
    data_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
)